In [1]:
import sys
sys.path.append(r'D:\CE7454_2018\VAE FOR STOCHASTIC INTEGER PROGRAMMING\CVAE for stochastic programming')
from local_stochoptim.stochprob.network_design import network_design_problem as ndp

In [2]:
import numpy as np
#import .local.stochoptim.stochprob.network_design.network_design_problem as ndp
import stochoptim.stochprob.network_design.network_design_solution as nds
import stochoptim.stochprob.network_design.network_design_uncertainty as ndu
from stochoptim.scenclust.cost_space_partition import CostSpaceScenarioPartitioning
import stochoptim.scengen.scenario_tree as st
np.set_printoptions(linewidth=120)

In [3]:
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset, TensorDataset
from collections import defaultdict

# for clustering
from sklearn_extra.cluster import KMedoids
#from pyclustering.cluster.kmedoids import kmedoids
import pickle

### Generate Problem Settings

In [4]:
n_origins = 2
n_destinations = 2
n_intermediates = 40
#=======================
n_scenarios = 200
distribution = 'uniform'
lb = 5
ub = 20
corr = 0.4

### No objectives

In [5]:
# from models import VAE
# torch.manual_seed(0)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # the net
# vae = VAE(
#     encoder_layer_sizes=[4, 256],
#     latent_size=2,
#     decoder_layer_sizes=[256, 4],
#     conditional=True,
#     num_labels=2 if True else 0).to(device)

# # Specify a path
# PATH = "trained_nets/saved_model_"+ str(1999) + ".pt"
# # load parameters
# vae.load_state_dict(torch.load(PATH))
# vae.eval()
############################################################GCN
from config import *
config_path = "configs/tsp10_ori.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
from models_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae = VAE(
    config=config,     
    encoder_layer_sizes=[4, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 4],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/saved_model_GCN_"+ str(100) + ".pt"
# load parameters
vae.load_state_dict(torch.load(PATH))
vae.eval()

Loaded configs/tsp10_ori.json:
{'expt_name': 'tsp10', 'gpu_id': '0', 'train_filepath': './data/tsp10_train_concorde.txt', 'val_filepath': './data/tsp10_val_concorde.txt', 'test_filepath': './data/tsp10_test_concorde.txt', 'num_nodes': 14, 'num_neighbors': -1, 'node_dim': 2, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 30, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=4, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=3, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=128, out_features=128, bias=True)
          (V): Linear(in_features=128, out_features=128, bias=True)
        )
        (edge_feat): EdgeFeatures(
          (

### With objectives

In [5]:
# from models_objs import VAE
# torch.manual_seed(0)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # the net
# vae_objs = VAE(
#     encoder_layer_sizes=[4, 256],
#     latent_size=2,
#     decoder_layer_sizes=[256, 4],
#     conditional=True,
#     num_labels=2 if True else 0).to(device)

# # Specify a path
# PATH = "trained_nets/ob_saved_model_"+ str(1999) + ".pt"
# # load parameters
# vae_objs.load_state_dict(torch.load(PATH))
# vae_objs.eval()
################################################### GCN
from config import *
config_path = "configs/tsp10.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
from models_objs_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae_objs = VAE(
    config=config,     
    encoder_layer_sizes=[4, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 4],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/ob_saved_model_GCN_"+ str(300) + ".pt"
# load parameters
vae_objs.load_state_dict(torch.load(PATH))
vae_objs.eval()

Loaded configs/tsp10.json:
{'expt_name': 'tsp10', 'gpu_id': '0', 'train_filepath': './data/tsp10_train_concorde.txt', 'val_filepath': './data/tsp10_val_concorde.txt', 'test_filepath': './data/tsp10_test_concorde.txt', 'num_nodes': 14, 'num_neighbors': -1, 'node_dim': 2, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 25, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
    (obj): Sequential(
      (L0): Linear(in_features=128, out_features=128, bias=True)
      (A0): ReLU()
    )
    (linear_obj): Linear(in_features=128, out_features=1, bias=True)
    (m_obj): Sigmoid()
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=4, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=3, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): N

### Generate instances

In [6]:
num_instances = 50

In [7]:
# param_list = []
# scenario_tree_list = []
# for _ in range(num_instances):    
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
# #     network_problem = ndp.NetworkDesign(param)
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=lb, 
#                                   ub=ub, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
#     param_list.append(param)
#     scenario_tree_list.append(scenario_tree)
# df=open('test_data_threshold_800.txt', 'wb')
# pickle.dump(param_list, df, -1)
# pickle.dump(scenario_tree_list, df, -1)
# df.close() 

In [8]:
# # dependeence between context and scenarios
# param_list = []
# scenario_tree_list = []
# for _ in range(num_instances):    
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
#     ran_s = param['opening_cost'].mean()
#     ran_s = (ran_s-3)/8*15+5  
#     ran_l = param['shipping_cost'].copy()
#     ran_l[ran_l>999] = 0   
#     ran_l = ran_l.mean()
#     ran_l = (ran_l-5)/6*15+5   
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=round(ran_s)-5, 
#                                   ub=round(ran_l)+7, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
#     param_list.append(param)
#     scenario_tree_list.append(scenario_tree)
# df=open('test_data.txt', 'wb')
# pickle.dump(param_list, df, -1)
# pickle.dump(scenario_tree_list, df, -1)
# df.close()    

In [9]:
dfr=open('test_data_threshold_120.txt', 'rb')
param_list = pickle.load(dfr)
scenario_tree_list = pickle.load(dfr)

In [10]:
#Define number of clusters
n_c = 20

our_objs = []
our_objs_objs = []
km_objs = []
optim = []

our_objs_t = []
our_objs_objs_t = []
km_objs_t = []
optim_t = []
for i in range(num_instances):
    # Instantiate Problem
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
    param = param_list[i]
    network_problem = ndp.NetworkDesign(param)
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=lb, 
#                                   ub=ub, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
    scenario_tree = scenario_tree_list[i]    
    
    ############################################################ 1. Clustering with nets
    with torch.no_grad():
        tensor_c = torch.from_numpy(np.array([param['opening_cost'],param['shipping_cost'],param['capacity']]))
        tensor_c = tensor_c.unsqueeze(0).repeat(200, 1, 1, 1)   ###n_scenarios
        ### notice! 0
        tensor_c[tensor_c>999] = 0
        tensor_c[tensor_c>99999] = 0
        ### notice! 20
        tensor_x = torch.from_numpy(scenario_tree.to_numpy()/20)
        ### 10 can be changed to any value > 1
        tensor_x = tensor_x.unsqueeze(1).repeat(1, 10, 1)
        x, y = tensor_x.to(device).float(), tensor_c.to(device).float() 
#         s_t = time.time()    
#         recon_x, mean, log_var, z, target, _, lat = vae(x, y)
#         X = z.detach().cpu().numpy()
#         #X = lat.detach().cpu().numpy()  
#         kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
#         repres =  kmedoids.cluster_centers_
#         indices= []
#         for j in range(repres.shape[0]):
#             indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
#         unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
#         if kmedoids.predict(repres)[0]!=unique_elements[0]:
#             counts_elements=counts_elements[::-1]
#         print(counts_elements) 
#         if len(counts_elements)<n_c:
#             continue
#         clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
#                                                          n_rvar={'d': 4},
#                                                          weights=np.array(counts_elements/counts_elements.sum()))  
#         solution_clustering_nets = network_problem.solve(clustering_nets, verbose=0)
#         our_objs_t.append(time.time()-s_t)        
#         stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['y']})
#         solution_nets = network_problem.solve(scenario_tree, 
#                                              decision_process=stage_0_decisions_nets, verbose=0)
#         our_objs.append(solution_nets.objective_value)   
    
    ###################################################################### 2. Clustering with nets with objectives
        s_t = time.time()    
        recon_x, mean, log_var, z, target, objs, _, lat = vae_objs(x[:,0,:], y)
        #X = torch.cat((z,objs),1).detach().cpu().numpy()
        X = z.detach().cpu().numpy()
        kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
        repres =  kmedoids.cluster_centers_
        indices= []
        for j in range(repres.shape[0]):
            indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
        unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
        if kmedoids.predict(repres)[0]!=unique_elements[0]:
            counts_elements=counts_elements[::-1]
        print(counts_elements)
        if len(counts_elements)<n_c:
#             our_objs.pop()
            continue        
        clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
                                                         n_rvar={'d': 4},
                                                         weights=np.array(counts_elements/counts_elements.sum()))    
        solution_clustering_nets = network_problem.solve(clustering_nets, verbose=0)
        our_objs_objs_t.append(time.time()-s_t)
        stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['y']})
        solution_nets = network_problem.solve(scenario_tree, 
                                             decision_process=stage_0_decisions_nets, verbose=0)
        our_objs_objs.append(solution_nets.objective_value)       
    ############################################################################# 3. start k-medoids
#     s_t = time.time()    
#     KX = scenario_tree.to_numpy()
#     Kkmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(KX)
#     Krepres =  Kkmedoids.cluster_centers_
#     Kindices= []
#     for j in range(Krepres.shape[0]):
#         Kindices.append(np.where(np.all(KX==Krepres[j], axis=1))[0][0])
#     Kunique_elements, Kcounts_elements = np.unique(Kkmedoids.labels_, return_counts=True)  
#     if Kkmedoids.predict(Krepres)[0]!=Kunique_elements[0]:
#         Kcounts_elements=Kcounts_elements[::-1]
#     print(Kcounts_elements) 
#     if len(Kcounts_elements)<n_c:
#         our_objs.pop()
#         our_objs_objs.pop()
#         continue      
#     Kclustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(Kindices)], 
#                                                      n_rvar={'d': 4},
#                                                      weights=np.array(Kcounts_elements/Kcounts_elements.sum())) 
#     Ksolution_clustering_nets = network_problem.solve(Kclustering_nets, verbose=0)
#     km_objs_t.append(time.time()-s_t)    
#     Kstage_0_decisions_nets = Ksolution_clustering_nets.decision_process_fixed({0: ['y']})
#     Ksolution_nets = network_problem.solve(scenario_tree, 
#                                          decision_process=Kstage_0_decisions_nets, verbose=0)
#     km_objs.append(Ksolution_nets.objective_value)
    ############################################################################### 4. actual solutions
#     s_t = time.time()     
#     actual_solution = network_problem.solve(scenario_tree, verbose=0)
#     optim_t.append(time.time()-s_t)       
#     optim.append(actual_solution.objective_value)
#     print(actual_solution.objective_value)
    print(str(i+1)+" instance(s) solved")

[14  9  9  8 13  5 14 16  6  6 12  4  4  6 14 10  9 19 10 12]
1 instance(s) solved
[ 2 18 14  1 14  2 13 15 15  2 14  3  4 10  5 18  9 20  8 13]
2 instance(s) solved
[ 6  2  8  3 14  1 14 21  5  4 14 10 11  9 17  6 16 18  8 13]
3 instance(s) solved
[ 4  7  6  2  8  2 11  4 15 22 17  5  9 17  6 11 10 16 18 10]
4 instance(s) solved
[ 9  6  8  5  5 13  5  7 10 11 14 17  6 10  6 11 18 16  6 17]
5 instance(s) solved
[ 9  5 19  7  9  7  3 15  2 10  1 15  7 14 19 15 12  8 14  9]
6 instance(s) solved
[ 7 14  6 15  9 18 13  9 13 20 10 13  3  3  6  7  4  5 14 11]
7 instance(s) solved
[15  5 16  8 17 21  4 13 11  2 13  7  8  7 10  9  7  2 12 13]
8 instance(s) solved
[ 7  2  5 10  3 12  5 15  9 11  7 13 10 17  9 13 11 10 16 15]
9 instance(s) solved
[ 3 12  1  4  1 13  5  9 20 11  8 20 14  4 16  5 13 12 16 13]
10 instance(s) solved
[14 14  4  3  6 10 12 16  6  6  7  6 11 11  7 14 23 13 11  6]
11 instance(s) solved
[ 9  8  6  4  6  5 10  1  7  4  8  9 23 11  9 12 18 18 16 16]
12 instance(s) solved
[

In [12]:
# optim = np.array([671.79875, 584.76375, 592.3875 , 632.18625, 706.40875, 642.78125, 594.82875, 706.5325 , 677.5775 , 646.97   ,
#        612.32125, 691.60625, 628.3575 , 627.3875 , 596.83875, 642.745  , 650.75625, 661.86625, 659.56625, 612.7875 ,
#        626.6875 , 669.49375, 620.65   , 677.44   , 660.93375, 586.58625, 648.15   , 636.6475 , 645.165  , 710.21125,
#        636.3925 , 638.0975 , 565.96   , 647.53125, 708.8525 , 602.24375, 666.88   , 616.52125, 601.1725 , 632.1625 ,
#        592.2825 , 626.8125 , 641.2025 , 652.67375, 673.0025 , 623.0475 , 589.6225 , 642.34875, 634.1425 , 653.89125]) #800

In [11]:
optim = np.array([579.85 , 577.35 , 550.755, 584.43 , 591.335, 561.385, 571.245, 579.335, 571.255, 596.57 , 563.89 , 571.99 ,
       577.87 , 562.705, 593.71 , 566.855, 580.24 , 561.165, 562.96 , 614.075, 559.98 , 564.54 , 577.13 , 571.93 ,
       590.155, 622.85 , 547.8  , 567.15 , 574.54 , 590.515, 566.135, 586.75 , 596.29 , 576.6  , 596.005, 572.435,
       587.975, 563.405, 557.9  , 551.82 , 548.04 , 563.7  , 582.015, 584.44 , 568.36 , 616.545, 582.99 , 543.015,
       549.405, 547.41 ]) #40 

In [12]:
((np.array(our_objs)-np.array(optim))/np.array(optim)).mean()  ####1000 with gcn

ValueError: operands could not be broadcast together with shapes (0,) (50,) 

In [13]:
((np.array(km_objs)-np.array(optim))/np.array(optim)).mean()   ####1000 

ValueError: operands could not be broadcast together with shapes (0,) (50,) 

In [14]:
((np.array(our_objs_objs)-np.array(optim))/np.array(optim)).mean()   ####1000 with gcn

0.03415855818819076

In [15]:
np.array(our_objs).mean(), np.array(our_objs).std()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid

(nan, nan)

In [16]:
np.array(km_objs).mean(), np.array(km_objs).std()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.


(nan, nan)

In [17]:
np.array(our_objs_objs).mean(), np.array(our_objs_objs).std()

(594.1148000000001, 44.61205264230729)

In [20]:
np.array(optim).mean(), np.array(optim).std()

(574.5359, 17.713244583926464)

In [20]:
np.array(our_objs_t).mean()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.


nan

In [21]:
np.array(km_objs_t).mean()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.


nan

In [22]:
np.array(our_objs_objs_t).mean()

7.4466258144378665

In [24]:
np.array(optim_t).mean()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.


nan

In [22]:
np.array(optim)

array([579.85 , 577.35 , 550.755, 584.43 , 591.335, 561.385, 571.245, 579.335, 571.255, 596.57 , 563.89 , 571.99 ,
       577.87 , 562.705, 593.71 , 566.855, 580.24 , 561.165, 562.96 , 614.075, 559.98 , 564.54 , 577.13 , 571.93 ,
       590.155, 622.85 , 547.8  , 567.15 , 574.54 , 590.515, 566.135, 586.75 , 596.29 , 576.6  , 596.005, 572.435,
       587.975, 563.405, 557.9  , 551.82 , 548.04 , 563.7  , 582.015, 584.44 , 568.36 , 616.545, 582.99 , 543.015,
       549.405, 547.41 ])

### Solutions to 800 scenarios

In [ ]:
np.array([608.4475 , 602.58875, 668.53375, 612.94   , 623.90625, 642.61875, 673.07   , 636.12875, 634.08125, 671.725  ,
       609.325  , 658.70375, 606.95125, 693.46125, 624.515  , 632.16125, 618.255  , 683.27875, 618.635  , 634.3875 ,
       657.82625, 665.75375, 590.38875, 610.29625, 651.425  , 647.075  , 673.915  , 618.3325 , 642.2975 , 601.49625])

### Solutions to size 44 

In [ ]:
array([589.255, 605.045, 601.2  , 594.075, 562.77 , 557.11 , 552.66 , 562.815, 592.395, 571.12 , 575.38 , 550.145,
       564.015, 580.235, 558.785, 582.285, 615.625, 556.265, 577.905, 550.715, 576.935, 596.91 , 573.825, 579.905,
       568.37 , 560.325, 550.91 , 583.005, 573.925]) #### 3